# SUMMA workflow: make folder structure
Makes the initial folder structure for a given control file. All other files in the workflow will look for the file `control_active.txt` during their execution. This script:

1. Copies the specified control file into `control_active.txt`;
2. Prepares a folder structure using the settings in `control_active.txt`.
3. Creates a copy of itself to be stored in the new folder structure.

The destination folders are referred to as "domain folders".

In [1]:
# Specify the control file to use
sourceFile  = 'control_Bow_at_Banff.txt'

Do not change below this line.

In [3]:
# Modules
import os
from pathlib import Path
from shutil import copyfile
from datetime import datetime

#### Copy the control file into `control_active.txt`

In [9]:
# Easy access to control file folder
controlFolder = Path('../0_controlFiles')

In [10]:
# Store the name of the 'active' file in a variable
controlFile = 'control_active.txt'

In [11]:
copyfile( controlFolder/sourceFile, controlFolder/controlFile );

#### Create the main domain folders

In [12]:
# Function to extract a given setting from the control file
def read_from_control( file, setting ):
    
    # Open 'control_active.txt' and ...
    for line in open(file):
        
        # ... find the line with the requested setting
        if setting in line:
            break
    
    # Extract the setting's value
    substring = line.split('|',1)[1]      # Remove the setting's name (split into 2 based on '|', keep only 2nd part)
    substring = substring.split('#',1)[0] # Remove comments, does nothing if no '#' is found
    substring = substring.strip()         # Remove leading and trailing whitespace, tabs, newlines
    
    # Return this value    
    return substring

In [13]:
# Find the path where the domain folders need to go
# Immediately store as a 'Path' to avoid issues with '/' and '\' on different operating systems
rootPath = Path( read_from_control(controlFolder/controlFile,'root_path') )

In [14]:
# Find the domain name
domainName = read_from_control(controlFolder/controlFile,'domain_name')

In [15]:
# Create the domain folder inside 'root'
domainFolder = 'domain_' + domainName
Path( rootPath / domainFolder ).mkdir(parents=True, exist_ok=True)

In [16]:
# Create a folder where the user can place shapefiles
Path( rootPath / domainFolder / 'shapefiles/catchment' ).mkdir(parents=True, exist_ok=True)
Path( rootPath / domainFolder / 'shapefiles/river_network' ).mkdir(parents=True, exist_ok=True)

#### Code provenance
Generates a basic log file in the domain folder and copies the control file and itself there.

In [17]:
# Create a log folder
logFolder = '_log'
Path( rootPath / domainFolder / logFolder ).mkdir(parents=True, exist_ok=True)

In [19]:
# Copy the control file
copyfile(controlFolder / sourceFile, rootPath / domainFolder / logFolder / sourceFile);

In [20]:
# Copy this script
thisFile = 'make_folder_structure.ipynb'
copyfile(thisFile, rootPath / domainFolder / logFolder / thisFile);

In [21]:
# Get current date and time
now = datetime.now()

In [23]:
# Create a log file 
logFile = now.strftime('%Y%m%d') + '_log.txt'
with open(rootPath / domainFolder / logFolder / logFile, 'w') as file:
    
    lines = ['Log generated by ' + thisFile + ' on ' + now.strftime('%Y/%m/%d %H:%M:%S') + '\n',
             'Generated folder structure using ' + sourceFile]
    for txt in lines:
        file.write(txt)    